In [19]:
from epigenomic_dataset import load_epigenomes
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler

# The considered window size
window_size = 200

# Retrieving the input data
X, y = load_epigenomes(
    cell_line = "HEK293",
    dataset = "fantom",
    regions = "enhancers",
    window_size = window_size
)

y = y.values.ravel()

# Imputation of NaN Values
X[X.columns] = KNNImputer(n_neighbors=X.shape[0]//10).fit_transform(X)

# Robust normalization of the values
X[X.columns] = RobustScaler().fit_transform(X)

X = X.values
shape=X.shape[1]

# Here one should feature selection. How can we do this?

In [1]:
from typing import Dict, Tuple, List

class Model:
    def __init__(self, name: str, model, **kwargs):
        self.name = name
        self.model = model
        self.kwargs = kwargs
        
    def __str__(self) -> str:
        return self.name
    
    def __repr__(self) -> str:
        return self.name
    
    def get_model(self) -> Tuple:
        return (self.model, self.kwargs)

In [2]:
from sklearn.tree import DecisionTreeClassifier

def get_decision_tree(name: str = 'DecisionTree', criterion: str = 'gini', max_depth: int = 50, 
                      random_state: int = 42, class_weight: str = 'balanced', **kwargs) -> Model:
    model = DecisionTreeClassifier(
        criterion=criterion,
        max_depth=max_depth,
        random_state=random_state,
        class_weight=class_weight
    )
    return Model(name, model, **kwargs)

Model.DecisionTree = get_decision_tree

In [3]:
Model.DecisionTree()

DecisionTree

In [4]:
from sklearn.ensemble import RandomForestClassifier
from multiprocessing import cpu_count


def get_random_forest(name: str = 'RandomForest', n_estimators: int = 500, criterion: str = 'gini', 
                      max_depth: int = 30, random_state: int = 42, 
                      class_weight: str = 'balanced', n_jobs: int = cpu_count, **kwargs):
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        criterion=criterion,
        max_depth=max_depth,
        random_state=random_state,
        class_weight=class_weight,
        n_jobs=n_jobs
    )
    return Model(name, model, **kwargs)

Model.RandomForest = get_random_forest

In [5]:
Model.RandomForest()

RandomForest

In [6]:
from tensorflow.keras.layers import Layer, Input, Flatten, Reshape, Dense, Conv2D, BatchNormalization, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import AUC
from sklearn.model_selection import StratifiedShuffleSplit


In [34]:
from tensorflow.keras.layers import AlphaDropout, ThresholdedReLU

In [ ]:
from tensorflow.keras.models import Sequential

def get_sequential(default_name: str = 'Sequential'):
    # no first and last layer
    def get_layers(*layers: Tuple[Layer]):
        def get_model(input_shape: Tuple[int], name: str = None, optimizer: str = 'nadam', 
                      loss: str = 'binary_crossentropy', metrics: List = None,
                      epochs: int = 1000, batch_size: int = 1024, 
                      validation_split: float = 0.1, shuffle: bool = True, verbose: bool = False, 
                      callbacks: List = None, **kwargs):
            name = name or default_name
            input_layer = Input(shape=input_shape)
            output_layer = Dense(1, activation="sigmoid")
            model = Sequential((input_layer,) + layers + (output_layer,), name)
            
            matrics = metrics or [
                "accuracy",
                AUC(curve="ROC", name="auroc"),
                AUC(curve="PR", name="auprc")
            ]
            model.compile(
                optimizer=optimizer,
                loss=loss,
                metrics=metrics
            )
            
            kwargs.update({
                'epochs': epochs,
                'batch_size': batch_size,
                'validation_split': validation_split,
                'shuffle': shuffle,
                'verbose': verbose,
                'callbacks': callbacks
            })
            model.summary()
            return Model(name, model, **kwargs)
        return get_model
    return get_layers

Model.Sequential = get_sequential()

In [8]:
Model.Sequential()((104,20))

Model: "Sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 104, 1)            21        
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________


Sequential

In [9]:
Model.Perceptron = get_sequential('Perceptron')()

In [10]:
Model.Perceptron((104,))

Model: "Perceptron"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 105       
Total params: 105
Trainable params: 105
Non-trainable params: 0
_________________________________________________________________


Perceptron

In [11]:
Model.MLP = get_sequential('MLP') 
Model.FFNN = get_sequential('FFNN')
Model.CNN = get_sequential('CNN')

In [41]:
#training 20 sec 
Model.MLP_Epi = Model.MLP(
    Dense(128, activation="relu"),
    Dense(64, activation="relu"),
    Dense(32, activation="relu"),
)

In [42]:
#training 40 sec
Model.FFNN_Epi = Model.FFNN(
    Dense(256, activation="relu"),
    Dense(128),
    BatchNormalization(),
    Activation("relu"),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(32, activation="relu"),
    Dense(16, activation="relu"),
)

In [43]:
#test con Alphadropout e ThresholdedReLU
Model.FFNN_Epi_2 = Model.FFNN(
    Dense(256, activation="relu"),
    Dense(128),
    BatchNormalization(),
    ThresholdedReLU(0.05),
    Dense(64, activation="relu"),
    AlphaDropout(0.3), #new
    Dense(32, activation="relu"),
    Dense(16, activation="relu"),
)

In [44]:
#test con
Model.FFNN_Epi_3 = Model.FFNN(
    Dense(256, activation="relu"),
    Dense(128, activation="relu"),
    BatchNormalization(),
    ThresholdedReLU(0.05),
    Dense(64, activation="relu"),
    Dense(64, activation="relu"),
    AlphaDropout(0.5), #new
    Dense(32, activation="relu"),
    Dense(16, activation="relu"),
)

In [15]:
Model.MLP_Seq = Model.MLP(
    Flatten()
)(input_shape=(200, 4))

Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 801       
Total params: 801
Trainable params: 801
Non-trainable params: 0
_________________________________________________________________


In [16]:
Model.FFNN_Seq = Model.FFNN(
    Flatten(),
    Dense(128, activation="relu"),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(32, activation="relu"),
    Dense(16, activation="relu"),
)(input_shape=(200, 4))

Model: "FFNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               102528    
_________________________________________________________________
dense_17 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 32)                2080   

In [17]:
Model.CNN_Seq = Model.CNN(
    Reshape((200, 4, 1)),
    Conv2D(64, kernel_size=(10, 2), activation="relu"),
    Conv2D(64, kernel_size=(10, 2), activation="relu"),
    Dropout(0.3),
    Conv2D(32, kernel_size=(10, 2), strides=(2, 1), activation="relu"),
    Conv2D(32, kernel_size=(10, 1), activation="relu"),
    Conv2D(32, kernel_size=(10, 1), activation="relu"),
    Dropout(0.3),
    Flatten(),
    Dense(32, activation="relu"),
    Dense(16, activation="relu"),
)(input_shape=(200, 4))

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 200, 4, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 191, 3, 64)        1344      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 182, 2, 64)        81984     
_________________________________________________________________
dropout_3 (Dropout)          (None, 182, 2, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 87, 1, 32)         40992     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 78, 1, 32)         10272     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 69, 1, 32)         10272   

In [24]:
from sklearn.model_selection import StratifiedShuffleSplit

splits = 50 #da testare un numero diverso di split
holdouts = StratifiedShuffleSplit(n_splits=splits, test_size=0.2, random_state=42)


In [26]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, average_precision_score
from sanitize_ml_labels import sanitize_ml_labels
import numpy as np

def report(y_true:np.ndarray, y_pred:np.ndarray)->np.ndarray:
    integer_metrics = accuracy_score, balanced_accuracy_score
    float_metrics = roc_auc_score, average_precision_score
    results1 = {
        sanitize_ml_labels(metric.__name__): metric(y_true, np.round(y_pred))
        for metric in integer_metrics
    }
    results2 = {
        sanitize_ml_labels(metric.__name__): metric(y_true, y_pred)
        for metric in float_metrics
    }
    return {
        **results1,
        **results2
    }

#si usano solo per categorizzazione, non hanno senso per regressione
# auroc: area sotto la curva di Receiver operating characteristic: analizza falsi positivi e negativi
# valore minimo è 0.5 (non impara niente)  valore massimo è 1 (modello perfetto)
# auprc: area sotto la curva di precision recall: va a computare per diversi treashold di precisioni diversi, la somma delle aree
#sottese sotto i diversi traingolini della precision-reacall. Un modello che non impara nulla ha AUPRC = 0 e un modello perfetto ha
# AUPRC = 1

In [28]:
def precomputed(results, model:str, holdout:int)->bool:
    df = pd.DataFrame(results)
    if df.empty:
        return False
    return (
        (df.model == model) &
        (df.holdout == holdout)
    ).any()

In [46]:
models=[]

models.append(Model.MLP_Epi((shape,),name="MLP"))
models.append(Model.FFNN_Epi((shape,),name="FFNN"))
models.append(Model.FFNN_Epi_2((shape,),name="FFNN_test1"))
models.append(Model.FFNN_Epi_3((shape,),name="FFNN_test2"))

Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_65 (Dense)             (None, 128)               26624     
_________________________________________________________________
dense_66 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_67 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_88 (Dense)             (None, 1)                 33        
Total params: 36,993
Trainable params: 36,993
Non-trainable params: 0
_________________________________________________________________
Model: "FFNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_68 (Dense)             (None, 256)               53248     
_____________________________________________

In [30]:
from tqdm.auto import tqdm
import pandas as pd

In [31]:
import json
import compress_json
#if os.path.exists("results1.json"):
 #   with open('results.json') as json_file:
#        results = json.load(json_file)
#else:
results = []
    
for i, (train, test) in tqdm(enumerate(holdouts.split(X, y)), total=splits, desc="Computing holdouts", dynamic_ncols=True):
    for model, params in tqdm([model.get_model() for model in models], total=len(models), desc="Training models", leave=False, dynamic_ncols=True):
        model_name = (
            model.__class__.__name__
            if model.__class__.__name__ != "Sequential"
            else model.name
        )
        if precomputed(results, model_name, i):
            continue
        model.fit(X[train], y[train], **params)
        results.append({
            "model":model_name,
            "run_type":"train",
            "holdout":i,
            **report(y[train], model.predict(X[train]))
        })
        results.append({
            "model":model_name,
            "run_type":"test",
            "holdout":i,
            **report(y[test], model.predict(X[test]))
        })
        compress_json.local_dump(results, "resultsPercHek.json")

Epoch 1/1000
46/46 [==============================] - 0s 5ms/step - loss: 0.8631 - val_loss: 0.8059
Epoch 2/1000
46/46 [==============================] - 0s 5ms/step - loss: 0.7265 - val_loss: 0.6996
Epoch 3/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.6417 - val_loss: 0.6338
Epoch 4/1000
46/46 [==============================] - 0s 3ms/step - loss: 0.5887 - val_loss: 0.5919
Epoch 5/1000
46/46 [==============================] - 0s 3ms/step - loss: 0.5533 - val_loss: 0.5618
Epoch 6/1000
46/46 [==============================] - 0s 4ms/step - loss: 0.5277 - val_loss: 0.5391
Epoch 7/1000
46/46 [==============================] - 0s 4ms/step - loss: 0.5084 - val_loss: 0.5211
Epoch 8/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.4934 - val_loss: 0.5071
Epoch 9/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.4813 - val_loss: 0.4951
Epoch 10/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.4711 - val_loss: 0.4849

46/46 [==============================] - 0s 2ms/step - loss: 0.3063 - val_loss: 0.3139
Epoch 163/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3062 - val_loss: 0.3140
Epoch 164/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3063 - val_loss: 0.3140
Epoch 165/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3062 - val_loss: 0.3141
Epoch 166/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3062 - val_loss: 0.3144
Epoch 167/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3062 - val_loss: 0.3140
Epoch 168/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3062 - val_loss: 0.3139
Epoch 169/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3063 - val_loss: 0.3146
Epoch 170/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3062 - val_loss: 0.3140
Epoch 171/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3062 - val_loss: 0.

46/46 [==============================] - 0s 2ms/step - loss: 0.3063 - val_loss: 0.3149
Epoch 323/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3062 - val_loss: 0.3149
Epoch 324/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3062 - val_loss: 0.3137
Epoch 325/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3062 - val_loss: 0.3140
Epoch 326/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3063 - val_loss: 0.3143
Epoch 327/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3063 - val_loss: 0.3147
Epoch 328/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3061 - val_loss: 0.3140
Epoch 329/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3062 - val_loss: 0.3141
Epoch 330/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3062 - val_loss: 0.3142
Epoch 331/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3062 - val_loss: 0.

46/46 [==============================] - 0s 2ms/step - loss: 0.3061 - val_loss: 0.3138
Epoch 483/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3061 - val_loss: 0.3140
Epoch 484/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3063 - val_loss: 0.3139
Epoch 485/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3064 - val_loss: 0.3144
Epoch 486/1000
46/46 [==============================] - 0s 3ms/step - loss: 0.3061 - val_loss: 0.3143
Epoch 487/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3062 - val_loss: 0.3146
Epoch 488/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3062 - val_loss: 0.3144
Epoch 489/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3063 - val_loss: 0.3149
Epoch 490/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3062 - val_loss: 0.3143
Epoch 491/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3063 - val_loss: 0.

46/46 [==============================] - 0s 3ms/step - loss: 0.3061 - val_loss: 0.3142
Epoch 643/1000
46/46 [==============================] - 0s 3ms/step - loss: 0.3063 - val_loss: 0.3142
Epoch 644/1000
46/46 [==============================] - 0s 3ms/step - loss: 0.3062 - val_loss: 0.3140
Epoch 645/1000
46/46 [==============================] - 0s 3ms/step - loss: 0.3062 - val_loss: 0.3146
Epoch 646/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3061 - val_loss: 0.3147
Epoch 647/1000
46/46 [==============================] - 0s 3ms/step - loss: 0.3061 - val_loss: 0.3143
Epoch 648/1000
46/46 [==============================] - 0s 3ms/step - loss: 0.3062 - val_loss: 0.3138
Epoch 649/1000
46/46 [==============================] - 0s 3ms/step - loss: 0.3063 - val_loss: 0.3139
Epoch 650/1000
46/46 [==============================] - 0s 3ms/step - loss: 0.3062 - val_loss: 0.3144
Epoch 651/1000
46/46 [==============================] - 0s 3ms/step - loss: 0.3063 - val_loss: 0.

...

46/46 [==============================] - 0s 2ms/step - loss: 0.3066 - val_loss: 0.3162
Epoch 163/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3066 - val_loss: 0.3161
Epoch 164/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3065 - val_loss: 0.3165
Epoch 165/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3066 - val_loss: 0.3163
Epoch 166/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3066 - val_loss: 0.3162
Epoch 167/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3065 - val_loss: 0.3157
Epoch 168/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3064 - val_loss: 0.3151
Epoch 169/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3066 - val_loss: 0.3159
Epoch 170/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3067 - val_loss: 0.3156
Epoch 171/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3065 - val_loss: 0.

46/46 [==============================] - 0s 2ms/step - loss: 0.3067 - val_loss: 0.3175
Epoch 323/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3066 - val_loss: 0.3155
Epoch 324/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3066 - val_loss: 0.3168
Epoch 325/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3065 - val_loss: 0.3155
Epoch 326/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3066 - val_loss: 0.3166
Epoch 327/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3065 - val_loss: 0.3157
Epoch 328/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3066 - val_loss: 0.3160
Epoch 329/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3066 - val_loss: 0.3163
Epoch 330/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3065 - val_loss: 0.3169
Epoch 331/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3066 - val_loss: 0.

Exception ignored in: <function ScopedTFFunction.__del__ at 0x000002E2C783BB88>
Traceback (most recent call last):
  File "C:\Users\matte\anaconda3\lib\site-packages\tensorflow\python\framework\c_api_util.py", line 96, in __del__
    self.deleter(self.func)
KeyboardInterrupt


46/46 [==============================] - 0s 3ms/step - loss: 0.3065 - val_loss: 0.3158
Epoch 392/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3065 - val_loss: 0.3152
Epoch 393/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3065 - val_loss: 0.3166
Epoch 394/1000
46/46 [==============================] - 0s 3ms/step - loss: 0.3065 - val_loss: 0.3154
Epoch 395/1000
46/46 [==============================] - 0s 3ms/step - loss: 0.3066 - val_loss: 0.3161
Epoch 396/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3067 - val_loss: 0.3159
Epoch 397/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3065 - val_loss: 0.3158
Epoch 398/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3067 - val_loss: 0.3156
Epoch 399/1000
46/46 [==============================] - 0s 3ms/step - loss: 0.3066 - val_loss: 0.3166
Epoch 400/1000
46/46 [==============================] - 0s 2ms/step - loss: 0.3065 - val_loss: 0.

KeyboardInterrupt: 